<a href="https://colab.research.google.com/github/Alessine/Alessine/blob/main/230326_nb1_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a id='Q0'></a>
<center> <h1> Notebook 1: Google Analytics 4 Data Analysis & Visualization</h1> </center>
<p style="margin-bottom:1cm;"></p>
<center><strong>Angela Niederberger, 2023</strong></center>
<p style="margin-bottom:1cm;"></p>

<div style="background:#EEEDF5;border-top:0.1cm solid #EF475B;border-bottom:0.1cm solid #EF475B;">
    <div style="margin-left: 0.5cm;margin-top: 0.5cm;margin-bottom: 0.5cm;color:#303030">
        <p><strong>Goal:</strong> Analyse Website Data from GA4 and Create Compelling Visuals</p>
        <strong> Outline:</strong>
        <a id='P0' name="P0"></a>
        <ol>
            <li> <a style="color:#303030" href='#I'>Introduction </a> </li>
            <li> <a style="color:#303030" href='#SU'>Set up</a></li>
            <li> <a style="color:#303030" href='#P1'>First Subset: Event Data</a></li>
            <li> <a style="color:#303030" href='#P2'>Part 2</a></li>
            <li> <a style="color:#303030" href='#P3'>Part 3</a></li>
            <li> <a style="color:#303030" href='#CL'>Conclusion</a></li>
        </ol>
        <strong>Keywords: BigQuery, Google Analytics 4, SQL, LookerStudio</strong>
    </div>
</div>
</nav>

<a id='I' name="I"></a>
## [Introduction](#P0)

Google has made a GA4 dataset available from its own Merchandise Store. It can be accessed easily via BigQuery and serves well for explorations of the general structure of GA4 data. The analysis and visualizations here are an opportunity to practice using this data with smaller subsets, but should also provide some interesting insights into this dataset. 

As a next step, the goal will be to perform the same analysis tasks directly in BigQuery (using SQL) and to recreate the visualizations in a LookerStudio dashboard. This would enable the processing of much larger amounts of data.

<a id='SU' name="SU"></a>
## [Set up](#P0)

### Packages

In [ ]:
# Google Cloud Libraries
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

# Wrangling
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

### Authorization

In [ ]:
auth.authenticate_user()
print('Authenticated')

Authenticated


### Magic Commands

In [ ]:
%matplotlib inline
%config Completer.use_jedi = False

### Global Parameters Setting

### User-Dependent Variables

## First Subset: Event Data

The new GA4 Data Model is entirely event-based. Therefore, the first subset of the data that I'm looking into just contains information on events.

### Data Loading

I'm using the library `google.cloud.bigquery` - check the [documentation](https://cloud.google.com/python/docs/reference/bigquery/latest/index.html) for more information.

In [ ]:
project = 'web-analytics-374017' # My project ID
location = 'US' # Dataset Location
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()

In [ ]:
# SQL query used to call data from BigQuery:
# Event data in January 2021
sql_query = """
  SELECT
    event_date,
    event_timestamp,
    event_name,
    event_params.key AS event_param_key,
    event_params.value.*,
    user_pseudo_id,
  FROM
    `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  CROSS JOIN
    UNNEST(event_params) AS event_params
  WHERE event_date BETWEEN "20210101"
  AND "20210131"
"""

df_events = client.query(sql_query).to_dataframe()

In [ ]:
df_events.head()

,event_date,event_timestamp,event_name,event_param_key,string_value,int_value,float_value,double_value,user_pseudo_id
0,20210125,1611568466723750,view_promotion,page_title,Home,<NA>,NaN,NaN,61243398.6312559948
1,20210125,1611614098807421,select_promotion,session_engaged,1,<NA>,NaN,NaN,62675582.3881827967
2,20210125,1611614098807421,view_promotion,session_engaged,1,<NA>,NaN,NaN,62675582.3881827967
3,20210125,1611533240927791,view_promotion,page_location,https://shop.googlemerchandisestore.com/,<NA>,NaN,NaN,63326662.4577590696
4,20210125,1611609917093305,view_promotion,engagement_time_msec,None,126,NaN,NaN,66749267.2079297955


### Data Exploration

#### Number of Pageviews per Day

In [ ]:
df_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11762784 entries, 0 to 11762783
Data columns (total 9 columns):
 #   Column           Dtype  
---  ------           -----  
 0   event_date       object 
 1   event_timestamp  Int64  
 2   event_name       object 
 3   event_param_key  object 
 4   string_value     object 
 5   int_value        Int64  
 6   float_value      float64
 7   double_value     float64
 8   user_pseudo_id   object 
dtypes: Int64(2), float64(2), object(5)
memory usage: 830.1+ MB


In [ ]:
df_events["event_name"].value_counts()

page_view              4377354
user_engagement        2724233
scroll                 1532382
view_item               839774
session_start           667820
first_visit             586287
view_promotion          458172
add_to_cart             141852
begin_checkout          131525
select_item             103785
view_search_results      83655
add_shipping_info        38727
add_payment_info         27922
select_promotion         25106
purchase                 21174
click                     2945
view_item_list              71
Name: event_name, dtype: int64

In [ ]:
df_events["event_param_key"].value_counts()

ga_session_number        1210147
page_location            1210147
ga_session_id            1210147
page_title               1202594
engaged_session_event    1171598
session_engaged          1113643
debug_mode               1004702
page_referrer             964706
engagement_time_msec      740021
medium                    336342
campaign                  336340
source                    335200
all_data                  213369
clean_event               213355
percent_scrolled          138997
entrances                 109558
term                      102623
gclid                      58176
gclsrc                     58018
currency                    7997
search_term                 7815
unique_search_term          6225
dclid                       4696
payment_type                 904
transaction_id               904
value                        904
tax                          904
shipping_tier                898
coupon                       776
promotion_name               425
outbound  

<a id='P2' name="P2"></a>
## [Part 2 title here](#P0)

<a id='CL'></a>
## [Conclusion](#P0)

This is a pre-written conclusion in which we have nice figures.

<div style="border-top:0.1cm solid #EF475B"></div>
    <strong><a href='#Q0'><div style="text-align: right"> <h3>End of this Notebook.</h3></div></a></strong>